<h1 align="center">Value Normalization: An Introduction</h1>
<h2 align="center">Adel Ardalan</h2>
<h3 align="center">With Walter Cai and AnHai Doan</h3>
<h4 align="center">University of Wisconsin-Madison</h4>

# Motivation

(Attribute) value normalization is one of the important steps in the data cleaning pipelines.

Suppose you are given the following table to analyze the sales of different products:

| Title | Brand | Color | Price | ... |
|---------------|---------------|---------------|---------------|-----|
| Sony 55-inch Smart LED TV | Sony | black | 799.99 | ... |
| VIZIO M50 LED TV | VIZIO | Charcoal | 849.99 | ... |
| Panasonic VIERA Plasma | Panasonic | BLK | 849.99 | ... |
| Samsung 32" HDTV | Samsung | Silver | 729.99 | ... |
| ... | ... | ... | ... | ... |

# Motivation (Cont.)

Multiple values in a single column might refer to the same real-world entity.
  - In the table above, *black*, *Charcoal* and *BLK* refer to the same color. 

This might hurt the effectiveness of various data analysis tasks.
  - For example if you're looking for the sales of all black TVs, you might not get the results of items with colors  *Charcoal* or *BLK*.

Furthermore as more data comes in, we want to detect and resolve the above problem for attribute values of the new items.

# Motivation (Cont.)

To deal with this issue we *normalize* the attribute values. 
* First, for each value of the target attribute, we create a *normalization rule*. 
  * For the color attribute of our example table, we have the following normalization rules: 
    * Charcoal $\rightarrow$ Black
    * BLK $\rightarrow$ Black
    * black $\rightarrow$ Black
    * Silver $\rightarrow$ Silver
  * Each value on the left-hand-side of the the above rules is a value of the target attribute (here, the color attribute).
  * On the right-hand-side of each rule is the *canonical value* associated with the value on the LHS.

* Then we go through the rows of the table one by one and when the value of the target attribute matches the LHS of one of the rules, we replace it with the RHS of that rule.

# Motivation (Cont.)

The normalized table would look like the following:

| Title | Brand | Color | Price | ... |
|---------------|---------------|---------------|---------------|-----|
| Sony 55-inch Smart LED TV | Sony | Black | 799.99 | ... |
| VIZIO M50 LED TV | VIZIO | Black | 849.99 | ... |
| Panasonic VIERA Plasma | Panasonic | Black | 849.99 | ... |
| Samsung 32" HDTV | Samsung | Silver | 729.99 | ... |
| ... | ... | ... | ... | ... |

# Value Normalization Problem Definition

Given a set of input values $V = \{v_1, \dots, v_m\}$, we want to create a set of normalization rules $\mathcal{R} = \{v_j \rightarrow \tilde{v_j} | j = 1, \dots, m\}$ such that:
  - $\tilde{v_j}$ is the canonical value of $v_j$ and 
  - $\tilde{v_j} = \tilde{v_k}$ if and only if $v_j$ and $v_k$ refer to the same real-world entity.

# Need Human in the Loop

We can try to solve the above problem using synonym discovery methods.

However we argue that fully automated methods are often inaccurate and thus in real-world scenarios, a human needs to come in and clean up the results of such methods.

Hence the bottleneck of the real-world value normalization tasks would be the human effort.

We thus aim at reducing user effort when normalizing values.

# Minimum-Effort Value Normalization Problem Definition

Suppose a user $u$ wants to normalize a set of input values $V = \{v_1, \dots, v_m\}$. We want to create a normalization approach for $u$ to normalize $V$ such that:
  1. the precision and the recall of the normalization results are 1.0 and
  2. the effort spent by $u$ during the normalization process is minimized.

# Approaches to Value Normalization

1. Manual value normalization
2. Clustering-based value normalization

# Manual Value Normalization

**Naive approach**: a user goes through the values one by one and creates a normalization rule per value.
* Advantage:
  * Easy to understand
* Disadvantage: 
  * Extremely slow for even a few tens of values

**Better approach**: 
1. First the user groups the values into clusters of synonyms
2. Then she labels each group with the appropriate canonical value
3. Finally we create a rule per value which maps it to the label of the cluster is belongs to.

**Note**: For the rest of our discussion we ignore the labeling part since it is the same across all the normalization methods we study.

# Manual Value Normalization
A reasonable manual normalization approach consists of two main steps:
  1. Local-merging (short-term memory size = 3)
<div align="middle"><img style="width: 60%;" src="files/LocalMergeExample.png"></div>

# Manual Value Normalization (Cont.)
A reasonable manual normalization approach consists of two main steps:
  1. Local-merging
  2. Global-merging
<div align="middle"><img style="width: 60%;" src="files/GlobalMergeExample.png"></div>

# Manual Value Normalization (Cont.)

Normalization rules:
<div align="middle"><img style="width: 100%;" src="files/NormRulesExample.png"></div>

# Manual Value Normalization (Cont.)

* Advantages:
  * Fairly simple to understand
* Disadvantage:
  * Still slow

# Clustering-Based Value Normalization

* First we apply a clustering algorithm, such as hierarchical agglomerative clustering (HAC), to the attribute values.

<div align="middle"><img style="width: 30%; align: middle;" src="files/BadClusteringExample.png"></div>

# Clustering-Based Value Normalization (Cont.)

* Then the user goes through the resulting clusters one by one and for each cluster she splits the cluster if it is not pure; i.e. if it contains values referring to multiple entities.

<div align="middle"><img style="width: 30%; align: middle;" src="files/PureClusteringExample.png"></div>

* Finally she executes the local-merging and global-merging steps (as described before) on the clusters from the previous step.

# Clustering-Based Value Normalization (Cont.)

If the clusters generated by the clustering algorithm are mostly clean, then clustering-based normalization would significantly reduce user effort.

However in reality, clustering algorithms are far from perfect:
* They often produce big, messy clusters.
* These clusters are very hard to understand and clean up.
* Real-world analysts are known to fall back to manually normalizing values after having a hard time cleaning up clustering algorithm results.

# Our Key Idea

* Stop the clustering algorithm before it starts producing large and messy clusters

* We can do this by limiting the size of the clusters generated by the clustering algorithm.

* But what should be the maximum cluster size?

# Hybrid Machine-Human Value Normalization

* First for each possible value of the maximum cluster size $1 \leq \lambda \leq m$ we create *a normalization plan* which consists of two parts:
  1. *Machine part*: we run a modified HAC algorithm, called $\lambda$HAC, which limits the size of the output clusters by $\lambda$. The standard HAC parameters are set to default values.
  2. *Human part*: the user cleans up the results of the machine part by first splitting the impure clusters and then executing the two-stage manual normalization method, as described before. We develop a pseudo-code of the human part as a guideline for the user to clean up the results of the machine part.

# Hybrid Machine-Human Value Normalization (Cont.)

* We then develop a cost approximation method to approximate the effort user spends cleaning up the results of the machine part of each plan following the above pseudo-code.

* Next we enumerate the plans in the above space by changing the maximum cluster size $\lambda$. For each $P_{\lambda}$ we approximate the cost of the plan using our cost approximation method.

* Finally we pick the plan $P_{\lambda^*}$ with the lowest cost and execute it; i.e. run $\lambda$HAC with maximum cluster size $\lambda^*$ and then ask the user to clean up the resulting clusters.

# Demo Time!